<a href="https://colab.research.google.com/github/bartoszkozakiewicz/PARAGON_AI_BACKEND/blob/main/ParagonAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install easyocr
!pip install transformers
!pip install datasets

#1 OCR CONTENT

In [22]:
import easyocr
import cv2
from google.colab.patches import cv2_imshow
#For the first time it downloads the models for the languages chosen below.
#Not all languages are compatible with each other so you cannot put
#multiple languages below
#reader = easyocr.Reader(['hi', 'te', 'en'])  #Hindi, telugu, and English
#The above gives error that Telugu is only compatible with English.

#So let us just use Hindi and English
#To use GPU you need to have CUDA configured for the pytorch library.
reader = easyocr.Reader(['pl'], gpu=True)  #Hindi, telugu, and English

img = cv2.imread('/content/1.png')


In [23]:
results = reader.readtext(img, detail=1, paragraph=False) #Set detail to 0 for simple text output , paragraph=False

SHOW IMAGE WITH BOUNDING BOXES

In [ ]:
for (bbox, text, prob) in results:

    #Define bounding boxes
    (tl, tr, br, bl) = bbox
    tl = (int(tl[0]), int(tl[1]))
    tr = (int(tr[0]), int(tr[1]))
    br = (int(br[0]), int(br[1]))
    bl = (int(bl[0]), int(bl[1]))

    #Remove non-ASCII characters to display clean text on the image (using opencv)
    text = "".join([c if ord(c) < 128 else "" for c in text]).strip()

    #Put rectangles and text on the image
    cv2.rectangle(img, tl, br, (0, 255, 0), 2)
    cv2.putText(img, text, (tl[0], tl[1] - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
# show the output image
cv2_imshow(img)
cv2.waitKey(0)

# 2 START OF DATA PREPARATION

In [6]:
import pandas as pd
import numpy as np

prepared_results = []

for idx,result in enumerate(results):
  bbox = result[0][0] + result[0][2]
  prepared_results.append({"x0":int(bbox[0]),"y0":int(bbox[1]),"x2":int(bbox[2]),"y2":int(bbox[3]) , "word": result[1]})

data_to_json = prepared_results
dataframe = pd.DataFrame(prepared_results, dtype=np.int16)
dataframe.head(5)

<ipython-input-6-77bac07d40da>:11: FutureWarning: Could not cast to int16, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  dataframe = pd.DataFrame(prepared_results, dtype=np.int16)


,x0,y0,x2,y2,word
0,223,33,265,53,a
1,332,24,380,48,'
2,75,45,113,59,
3,114,39,223,60,6' !
4,111,59,157,75,4-


In [12]:
import json
# Write the data to a JSON file
with open('output.json', 'w') as json_file:
    json.dump(data_to_json, json_file, indent=4)

**PODZIAŁ**
- PRODUCT
- PRODUCT_PRICE
- AMOUNT

- SUM_PRICE
- COMPANY
- DATE
- O

- DISC
- DISC_PRICE

- ADDRESS


# 3 MODEL PREPARATION

### 3.2 Take tokenizers etc from hugginface - MODEL

In [ ]:
from transformers import (
    LayoutLMTokenizer,
)
import torch
from datasets import load_dataset, Features


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer = LayoutLMTokenizer.from_pretrained(
   "microsoft/layoutlm-base-uncased",
)

### 3.3 Preprocess data

#### 3.3.1 Normalize bbox

**TEST DATA**

In [273]:
data = {
    "words":[["a",":","asd","jan"],["a",":","hop","bart","jol"]],
    "bboxes":[[[1133,1286,1657,1366],[728,1350,1283,1448],[1701,1347,2058,1438],[723,1428,1029,1518]],[[933,786,1626,871],[836,840,1804,938],[1078,907,1497,988],[983,968,1559,1054],[925,1046,1007,1119]]],
    "labels":[[1,1,0,2],[0,1,2,2,1]]
}

In [ ]:
from PIL import Image
img_path = "/content/1.jpg"
image = Image.open(name_of_your_document).convert("RGB")

width, height = image.size

def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]

In [274]:
for i,boxes in enumerate(data["bboxes"]):
  for j,box in enumerate(boxes):
    box = normalize_bbox(box, 2500, 2500)
    data["bboxes"][i][j] = box


In [265]:
data

{'words': [['a', ':', 'asd', 'jan'], ['a', ':', 'hop', 'bart', 'jol']],
 'bboxes': [[[453, 514, 662, 546],
   [291, 540, 513, 579],
   [680, 538, 823, 575],
   [289, 571, 411, 607]],
  [[373, 314, 650, 348],
   [334, 336, 721, 375],
   [431, 362, 598, 395],
   [393, 387, 623, 421],
   [370, 418, 402, 447]]],
 'labels': [[1, 1, 0, 2], [0, 1, 2, 2, 1]]}

#### 3.3.2 Prepare dataset for model to train

In [275]:
def final_data_preparation(data,max_length=512):
  final_bbox = []
  final_labels = []
  for word,box,label in zip(data["words"],data["bboxes"],data["labels"]):
    tokenized_word = tokenizer.tokenize(word)
    # print("TOK_WORD",tokenized_word)
    #Prepare final bbox
    final_bbox.extend([box]*len(tokenized_word))
    # print("final_bbox",final_bbox)

    #Prepare final labels
    final_labels.append(label)
    # final_labels.extend([-1 for _ in range(len(tokenized_word)-1)]) # ??
    final_labels.extend([label for _ in range(len(tokenized_word)-1)])

  num_special_tokens = 2
  if len(final_bbox) > max_length - num_special_tokens:
    final_bbox = final_bbox[:(max_length-num_special_tokens)]
    final_labels = final_labels[:(max_length-num_special_tokens)]

  #Lastly make usage of special tokens in bbox and labels
  final_bbox =  [[0, 0, 0, 0]] + final_bbox + [[1000, 1000, 1000, 1000]]
  final_labels =  [0] + final_labels + [0]

  #Create encoding to get - input_ids, attention_mask, token_type_ids
  encoding = tokenizer(" ".join(data["words"]),padding="max_length",truncation=True)
  # print("ENCODING",encoding)
  # print("JOIN"," ".join(data["words"]))

  #If sentence was smaller than max_length then it have to be padding added to final_box and final_labels to keep compatibility
  pad_token_box = [0, 0, 0, 0]
  padding_length = max_length - len(tokenizer(' '.join(data["words"]), truncation=True)["input_ids"])
  final_bbox += [pad_token_box] * padding_length
  final_labels += [0] * padding_length


  ## Fill encoding with previously prepared bboxes and labels
  encoding['bboxes'] = final_bbox
  encoding['ner_tags'] = final_labels

  return encoding

#Process of preparing data
data_result=[]
for x in zip(data["words"], data["bboxes"], data["labels"]):
    x = {"words":x[0],"bboxes":x[1],"labels":x[2]}
    data_result.append(final_data_preparation(x))


Get data into torch dataloaders for training purposes

In [276]:
from torch.utils.data import Dataset, DataLoader

class LayoutLMDataset(Dataset):
    def __init__(self, data_results):#input_ids,bboxes, attention_mask, token_type_ids,labels
        self.data_results = data_results
        # self.input_ids = input_ids
        # self.bboxes=bboxes
        # self.attention_mask = attention_mask
        # self.token_type_ids = token_type_ids
        # self.labels = labels

    def __len__(self):
        return len(self.data_results)

    def __getitem__(self, idx):
        input_id = self.data_results[idx]["input_ids"]
        bbox = self.data_results[idx]["bboxes"]
        attention_mask = self.data_results[idx]["attention_mask"]
        token_type_id = self.data_results[idx]["token_type_ids"]
        label = self.data_results[idx]["ner_tags"]

        return {
            "input_ids": torch.tensor(input_id,dtype=torch.int64).flatten(),
            "bboxes":torch.tensor(bbox,dtype=torch.int64),
            "attention_mask": torch.tensor(attention_mask,dtype=torch.int64).flatten(),
            "token_type_ids":torch.tensor(token_type_id,dtype=torch.int64).flatten(),
            "ner_tags": torch.tensor(label,dtype=torch.int64).flatten()
        }

Prepare dataloader

In [277]:
lmDataset = LayoutLMDataset(data_result)
dataloader = DataLoader(lmDataset,batch_size=1, shuffle=True)

In [ ]:
batch = next(iter(dataloader))
inputs= batch
inputs

#### 3.3.3 Train/test split

In [279]:
train_dataloader=dataloader
valid_dataloader=dataloader

#4 TRAINING PROCEDURE

In [280]:
from tqdm.notebook import tqdm

for batch in tqdm(train_dataloader):
  labels = batch["ner_tags"].to(device).shape
  input_ids=batch["input_ids"].shape
  bbox=batch["bboxes"].shape
  attention_mask=batch["attention_mask"].shape,
  token_type_ids=batch["token_type_ids"].shape,
  # print(batch)

  print(labels,input_ids,bbox,attention_mask,token_type_ids)

  0%|          | 0/2 [00:00<?, ?it/s]

torch.Size([1, 512]) torch.Size([1, 512]) torch.Size([1, 512, 4]) (torch.Size([1, 512]),) (torch.Size([1, 512]),)
torch.Size([1, 512]) torch.Size([1, 512]) torch.Size([1, 512, 4]) (torch.Size([1, 512]),) (torch.Size([1, 512]),)


In [281]:
from tqdm.notebook import tqdm
from transformers import (
    LayoutLMForTokenClassification,
    AdamW
)

model = LayoutLMForTokenClassification.from_pretrained(
    "microsoft/layoutlm-base-uncased",num_labels=3
    )
model.to(device)

optimizer = AdamW(model.parameters(), lr=4e-5)
num_epochs = 5


for epoch in range(num_epochs):
    print("Epoch:", epoch)
    training_loss = 0.0
    model.train()
    for batch in tqdm(train_dataloader):
        labels = batch["ner_tags"].to(device)
        print("hop",labels.shape)
        outputs = model(
            input_ids=batch["input_ids"].to(device),
            bbox=batch["bboxes"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            token_type_ids=batch["token_type_ids"].to(device),
            labels=labels
        )
        loss = outputs.loss

        training_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print("Training Loss:", training_loss / batch["input_ids"].shape[0])

    validation_loss = 0.0
    for batch in tqdm(valid_dataloader):
        labels = batch["ner_tags"].to(device)
        outputs = model(
            input_ids=batch["input_ids"].to(device), bbox=batch["bboxes"].to(device),
            attention_mask=batch["attention_mask"].to(device),
            token_type_ids=batch["token_type_ids"].to(device),
            labels=labels
        )
        loss = outputs.loss

        validation_loss += loss.item()

    print("Validation Loss:", validation_loss / batch["input_ids"].shape[0])

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0


  0%|          | 0/2 [00:00<?, ?it/s]

hop torch.Size([1, 512])
hop torch.Size([1, 512])
Training Loss: 1.5572057366371155


  0%|          | 0/2 [00:00<?, ?it/s]

...

Validation Loss: 0.05146443285048008
Epoch: 4


  0%|          | 0/2 [00:00<?, ?it/s]

hop torch.Size([1, 512])
hop torch.Size([1, 512])
Training Loss: 0.05023973248898983


  0%|          | 0/2 [00:00<?, ?it/s]

Validation Loss: 0.04788589850068092


In [ ]:
model.save_pretrained('saved_model/')

# 5 MAKING PREDICTIONS (INFERENCE)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LayoutLMForTokenClassification.from_pretrained("saved_model")
model.to(device);

In [ ]:
labels = []
id2label = {idx: val for idx, val in enumerate(labels)}


In [307]:
def normalize_bbox(bbox, width, height):
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]

def normalize_bboxes_in_data(img_data):
  for j,box in enumerate(img_data["bboxes"]):
    box = normalize_bbox(box, 2500, 2500)
    img_data["bboxes"][j] = box
  return img_data


### 5.1 Ocr work

In [308]:
#Ocr progress

# ...

#Data in format:

img_data = {
    "words":["a",":","asd","jan"],
    "bboxes":[[1133,1286,1657,1366],[728,1350,1283,1448],[1701,1347,2058,1438],[723,1428,1029,1518]]
}

#Normalize data
normalized_data = normalize_bboxes_in_data(img_data)


[[453, 514, 662, 546],
 [291, 540, 513, 579],
 [680, 538, 823, 575],
 [289, 571, 411, 607]]

### 5.2 LayoutLM prediction - for single image

In [312]:
def predict_layoutLM(img_data):
  end_boxes = []
  for word, box in zip(img_data["words"],img_data["bboxes"]):
    tokenized_word = tokenizer.tokenize(word)
    end_boxes.extend([box]*len(tokenized_word))

  end_boxes = [[0,0,0,0]] + end_boxes + [[1000,1000,1000,1000]]

  encoding = tokenizer(" ".join(img_data["words"]), return_tensors="pt")
  end_boxes = torch.tensor([end_boxes]).to(device)

  outputs = model(
    input_ids=encoding["input_ids"].to(device),
    bbox=end_boxes,
    attention_mask=encoding["attention_mask"].to(device),
    token_type_ids=encoding["token_type_ids"].to(device),
  )

  preds = torch.nn.functional.softmax(outputs.logits,dim=2).cpu().detach().numpy()
  preds_idx = preds.argmax(axis=2)[0]

  return preds_idx


predict_layoutLM(normalized_data)

array([0, 0, 0, 0, 0, 0, 0])